In [11]:
import os
import xlcalculator


from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from formula_visualiser import FormulaVisualiser


In [12]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_1'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [13]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [14]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [15]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [16]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [17]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [18]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [19]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [20]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [21]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [22]:
series_mapping = SeriesMapper.map_series(series_dict)

In [23]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [24]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]


In [25]:
for series in series_list:
    formula_1 = series.formulas[0]
    print(formula_1)
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_new = series_implementer.replace_range_nodes(formula_1_ast)

=B3
cell_range: B3
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=3, coordinate=None, value=None, value_type=None)]
=B4
cell_range: B4
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=4, coordinate=None, value=None, value_type=None)]
=Sheet1!B3+Sheet1!C3
Sheet1!B3
cell_range: B3:B3
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=3, coordinate=None, value=None, value_type=None)]
Sheet1!C3
cell_range: C3:C3
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=3, row=3, coordinate=None, value=None, value_type=None)]
=SUM(Sheet1!B3:D3)
Sheet1!B3:D3
cell_range: B3:D3
worksheet: Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None)
cell: [Cell(column=2, row=3, coordinate=None, value=None, value_type=None), Cell(column=3, row=3, coordinate=None, v

In [26]:
formula_1_new = SeriesImplementer.serialise_ast_to_formula(formula_1_ast_new)

In [27]:
formula_1_ast_new = FormulaParser.parse_formula(f'={formula_1_new}')

In [28]:
formula_visualiser = FormulaVisualiser()
ast_graph = formula_visualiser.visualise(formula_1_ast_new)
ast_graph.render('formula_1_ast_new')

'formula_1_ast_new.png'